<a href="https://colab.research.google.com/github/NLPetroni/assignment_two/blob/main/solution_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and downloads



In [1]:
import numpy as np
import pandas as pd
import sys
import os

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  %cd /content
  !rm -rf assignment_two &> /dev/null
  !git clone https://github.com/NLPetroni/assignment_two &> /dev/null
  %cd assignment_two
  sys.path.append(os.getcwd())
  !git clone https://gitlab.com/sasso-effe/nlp-assignment-data.git &> /dev/null
  !mv nlp-assignment-data/embedding_matrix.npy res/embedding_matrix.npy
  !rm -rf nlp-assignment-data
  !pip install wandb &> /dev/null
  !pip install torchinfo

In [5]:
import re
import math
from functools import reduce
from typing import List, Callable
import pickle

import wandb
import nltk
from nltk.corpus import stopwords
import torch
from torch import nn
from torchinfo import summary
from sklearn.metrics import f1_score, precision_score, recall_score

from src import utils

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
utils.download_data('dataset')
train_set = pd.read_csv("dataset/train_pairs.csv")
val_set = pd.read_csv("dataset/val_pairs.csv")
test_set = pd.read_csv("dataset/test_pairs.csv")

print(train_set.columns)
print("Total rows of the train set: {:d}".format(len(train_set)))
print("Total rows of the validation set: {:d}".format(len(val_set)))
print("Total rows of the test set: {:d}".format(len(test_set)))
print(train_set['Label'].value_counts())

Index(['Unnamed: 0', 'Claim', 'Evidence', 'ID', 'Label'], dtype='object')
Total rows of the train set: 121740
Total rows of the validation set: 7165
Total rows of the test set: 7189
SUPPORTS    89389
REFUTES     32351
Name: Label, dtype: int64


# Dataset pre-processing and conversion

In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\t-]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
BAD_SYMBOLS_RE = re.compile('(-LRB-)|(-RRB-)|(-LSB-)|(-RSB-)')
INSIDE_SQAURE_BRACKETS_RE = re.compile('(-LSB-).*?(-RSB-)')

try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

def remove_inside_square_brackets(text: str) -> str:
    return INSIDE_SQAURE_BRACKETS_RE.sub('', text)

def remove_bad_symbols(text: str) -> str:
    return BAD_SYMBOLS_RE.sub('', text)

def remove_final_tags(text: str) -> str:
   return re.sub('\.\t.*?$', '', text) 

def lower(text: str) -> str:
    """
    Transforms given text to lower case.
    Example:
    Input: 'I really like New York city'
    Output: 'i really like new your city'
    """

    return text.lower()

def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)

def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace('</br>', '')

def filter_out_uncommon_symbols(text: str) -> str:
    """
    Removes any special character that is not in the
    good symbols list (check regular expression)
    """

    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text: str) -> str:
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])


def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()

def split_text(text: str) -> List:
  return text.split()

PREPROCESSING_PIPELINE = [
                          remove_inside_square_brackets,
                          remove_bad_symbols,
                          lower,
                          remove_final_tags,
                          replace_special_characters,
                          filter_out_uncommon_symbols,
                          strip_text,
                          split_text
                          ]

# Anchor method

def text_prepare(text: str,
                 filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """

    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE

    return reduce(lambda txt, f: f(txt), filter_methods, text)

def full_preprocessing(dataset):
    # In the evidences there is an id at the beginning of the sequence which is
    # removed with the splice [:1]
    dataset['Evidence'] = dataset['Evidence'].apply(lambda txt: text_prepare(txt)[1:])
    dataset['Claim'] = dataset['Claim'].apply(lambda txt: text_prepare(txt))
    return dataset


## Padding

In [8]:
class Dataset(torch.utils.data.Dataset):
    """Simple dataset class to use dataloaders (batching) """
    def __init__(self, ids, claims, evidences, labels, majority_vote=False):
        self.ids = ids
        self.claims = claims
        self.evidences = evidences
        self.labels = torch.tensor(labels, dtype=torch.float)
        self.majority_vote = majority_vote
        self.indexes = list(set(self.ids))
    def __getitem__(self, idx):
        if self.majority_vote:
          item = [(self.claims[i], self.evidences[i], self.labels[i]) for i,id in enumerate(self.ids) if id==self.indexes[idx]]
        else:
          item = self.claims[idx], self.evidences[idx], self.labels[idx]        
        return item
    def __len__(self):
        return self.claims.shape[0]

In [11]:

def to_tensor_collate(batch):

    claims, evidences, labels = zip(*batch)
    return torch.as_tensor(claims), torch.as_tensor(evidences), torch.as_tensor(labels)

def pad_batch_collate(batch):
    """Used by DataLoader to pad each batch independently"""
    max_claim_len = int(np.quantile([len(claim) for (claim, evidence, target) in batch], 0.99))
    max_evidence_len = int(np.quantile([len(evidence) for (claim, evidence, target) in batch], 0.99))
    
    claims = []
    evidences = []
    targets = []
    for claim, evidence, target in batch:
        if len(claim) > max_claim_len:
            claims.append(claim[:max_claim_len])
        else:
            claims.append(([400000] * (max_claim_len - len(claim))) + claim)
        if len(evidence) > max_evidence_len:
            evidences.append(evidence[:max_evidence_len])
        else:
            evidences.append(([400000] * (max_evidence_len - len(evidence))) + evidence)
        targets.append(target)

    return torch.as_tensor(claims), torch.as_tensor(evidences), torch.as_tensor(targets)

def pad_vote_collate(batch):
    """
    Collate function for the dataloader, used for majority voting evaluation.
    Returns:
        A list of batches, where each batch contains all the couples claim-evidence for a single claim.
        The list is unpacked in the evaluation method and passed batch-by-batch to the model.
    """
    for elem in batch:
        batch_res = []
        max_evidence_len = int(np.max([len(evidence) for (claim, evidence, target) in elem]))
        claims = []
        evidences = []
        targets = []
        for claim, evidence, target in elem:
            if len(evidence) > max_evidence_len:
                evidences.append(evidence[:max_evidence_len])
            else:
                evidences.append(([400000] * (max_evidence_len - len(evidence))) + evidence)
            claims.append(claim)
            targets.append(target)
        batch_res.append(torch.as_tensor(claims), torch.as_tensor(evidences), torch.as_tensor(targets))

    return batch_res



## Glove

In [16]:
def __generate_voc_and_matrix(train_set, val_set, test_set):
  voc_evidence_train = [item for sublist in train_set['Evidence'] for item in sublist]
  voc_claim_train = [item for sublist in train_set['Claim'] for item in sublist]
  TRAIN_VOC = set(voc_evidence_train + voc_claim_train)
  voc_evidence_val = [item for sublist in val_set['Evidence'] for item in sublist]
  voc_claim_val = [item for sublist in val_set['Claim'] for item in sublist]
  VAL_VOC = set(voc_evidence_val + voc_claim_val)
  voc_evidence_test = [item for sublist in test_set['Evidence'] for item in sublist]
  voc_claim_test = [item for sublist in test_set['Claim'] for item in sublist]
  TEST_VOC = set(voc_evidence_test + voc_claim_test)

  inputs = train_set['Evidence'].tolist() + train_set['Claim'].tolist()
  glove_voc, embedding_matrix = utils.get_glove(number_token=False)
  vocabulary, embedding_matrix = utils.add_oov(glove_voc, TRAIN_VOC, embedding_matrix, inputs)
  inputs = val_set['Evidence'].tolist() + val_set['Claim'].tolist()
  vocabulary, embedding_matrix = utils.add_oov(vocabulary, VAL_VOC, embedding_matrix, inputs)
  inputs = test_set['Evidence'].tolist() + test_set['Claim'].tolist()
  vocabulary, embedding_matrix = utils.add_oov(vocabulary, TEST_VOC, embedding_matrix, inputs)

  with open("res/vocabulary.pkl", "wb") as file:
    pickle.dump(vocabulary, file)
  np.save("res/embedding_matrix.npy", embedding_matrix)
  return vocabulary, embedding_matrix


def get_voc_and_matrix(train_set, val_set, test_set):
    if os.path.exists("res/vocabulary.pkl"):
      print('The vocabulary is already present. Loading it...', end=' ')
      with open('res/vocabulary.pkl', 'rb') as f:
        vocabulary = pickle.load(f)
      print('Done!')
      if os.path.exists("res/embedding_matrix.npy"):
        print('The embedding matrix are already present. Loading it...', end=' ')
        embedding_matrix = np.load("res/embedding_matrix.npy", )
        print("Done!")
      else:
        print('The embedding matrix is NOT present. You can download it from https://gitlab.com/sasso-effe/nlp-assignment-data and put it in the res folder')
    else:
      print("The vocabulary and the embedding matrix are NOT present. Creating them...")
      vocabulary, embedding_matrix = __generate_voc_and_matrix(train_set, val_set, test_set)
      print("Vocabulary and embedding matrix created! Remember to download the generated files.")
    return vocabulary, embedding_matrix


## Tokenization

In [13]:
def tokenize(input: List, vocabulary) -> torch.Tensor:
  result = list(map(lambda x: vocabulary[x], input))
  return result

def detokenize(input: torch.Tensor, inverse_vocabulary) -> List:
  result = input.tolist()
  # FIXME sistemare computazione
  result = list(map(lambda x: inverse_vocabulary[x], result))
  return result

def full_tokenization(data, vocabulary):
    data['Evidence'] = data['Evidence'].map(lambda x: tokenize(x, vocabulary))
    data['Claim'] = data['Claim'].map(lambda x: tokenize(x, vocabulary))
    data['Label'] = data['Label'].map(lambda x: float(1.0) if x == 'SUPPORTS' else float(0.0))
    return data

# Putting everything together

In [17]:
def load_data(padding=False):
    utils.download_data('dataset')
    train_set = pd.read_csv("dataset/train_pairs.csv")
    val_set = pd.read_csv("dataset/val_pairs.csv")
    test_set = pd.read_csv("dataset/test_pairs.csv")

    # preprocessing
    train_set = full_preprocessing(train_set)
    val_set = full_preprocessing(val_set)
    test_set = full_preprocessing(test_set)

    if padding:
        evidence_lens = [len(te) for te in train_set['Evidence']] + [len(ve) for ve in val_set['Evidence']] + [len(te) for te in test_set['Evidence']]
        max_evidence_len = np.quantile(evidence_lens, 0.99, interpolation='nearest')

        claim_lens = [len(te) for te in train_set['Claim']] + [len(ve) for ve in val_set['Claim']] + [len(te) for te in test_set['Claim']]
        max_claim_len = np.quantile(claim_lens, 0.99, interpolation='nearest')

        tr_ev_lens = train_set['Evidence'].map(lambda e: len(e))
        tr_claim_lens = train_set['Claim'].map(lambda c: len(c))
        train_set = train_set.drop(train_set[(tr_ev_lens >= max_evidence_len) | (tr_claim_lens >= max_claim_len)].index).reset_index()
        va_ev_lens = val_set['Evidence'].map(lambda e: len(e))
        va_claim_lens = val_set['Claim'].map(lambda c: len(c))
        val_set = val_set.drop(val_set[(va_ev_lens >= max_evidence_len) | (va_claim_lens >= max_claim_len)].index).reset_index()
        te_ev_lens = test_set['Evidence'].map(lambda e: len(e))
        te_claim_lens = test_set['Claim'].map(lambda c: len(c))
        test_set = test_set.drop(test_set[(te_ev_lens >= max_evidence_len) | (te_claim_lens >= max_claim_len)].index).reset_index()

        train_set.loc[:, 'Evidence'] = train_set['Evidence'].map(lambda x: x + (['<PAD>'] * (max_evidence_len - len(x))))
        val_set.loc[:, 'Evidence'] = val_set['Evidence'].map(lambda x: x + (['<PAD>'] * (max_evidence_len - len(x))))
        test_set.loc[:, 'Evidence'] = test_set['Evidence'].map(lambda x: x + (['<PAD>'] * (max_evidence_len - len(x))))

        train_set.loc[:, 'Claim'] = train_set['Claim'].map(lambda x: x + (['<PAD>'] * (max_claim_len - len(x))))
        val_set.loc[:, 'Claim'] = val_set['Claim'].map(lambda x: x + (['<PAD>'] * (max_claim_len - len(x))))
        test_set.loc[:, 'Claim'] = test_set['Claim'].map(lambda x: x + (['<PAD>'] * (max_claim_len - len(x))))


    vocabulary, embedding_matrix = get_voc_and_matrix(train_set, val_set, test_set)

    # tokenization
    train_set = full_tokenization(train_set, vocabulary)
    val_set = full_tokenization(val_set, vocabulary)
    test_set = full_tokenization(test_set, vocabulary)

    return vocabulary, embedding_matrix, (train_set, val_set, test_set)

# Model definition

In [ ]:
# lambda layers

class LambdaLast(torch.nn.Module):
  def forward(self, x):
    return x[:,-1]

class LambdaAvg(torch.nn.Module):
  def forward(self, x):
    return torch.mean(x, dim=1)

class LambdaConcatenation(torch.nn.Module):
  def forward(self, x, y):
    return torch.cat((x, y), dim=1)

class LambdaSum(torch.nn.Module):
  def forward(self, x, y):
    return x + y

class LambdaMean(torch.nn.Module):
  def forward(self, x, y):
    return (x + y) / 2


In [ ]:
def get_binary_classifier(name:str,
                    w_in: int,
                    w_hidden: int,
                    n_layers=2
                          ) -> nn.Sequential:
    """Gets a sequential container with a linear+relu+linear classifier

    Args:
        name: the name prefix to append to each layer in the container.
        w_in: the number of the input features.
        w_hidden: the number of internal weights

    Returns: the created sequential.
    """
    assert n_layers >= 2, f'n_layers is {n_layers}, must be >= 2'
    container = nn.Sequential()
    container.add_module(f'{name}_fc1', nn.Linear(in_features=w_in, out_features=w_hidden))    
    container.add_module(f'{name}_ReLU', nn.ReLU())
    for i in range(n_layers-2):
      container.add_module(f'{name}_fc{i+2}', nn.Linear(in_features=w_hidden, out_features=w_hidden))
      container.add_module(f'{name}_ReLU', nn.ReLU())
    container.add_module(f'{name}_fc2', nn.Linear(in_features=w_hidden, out_features=1))
    return container

 
class RNNEncoder(torch.nn.Module):

  def __init__(self, input_size, hidden_size, num_layers, rnn_type='elman', output_fn='last', verbose=False):
    super().__init__()
    if verbose:
      print('Initializing RNNEncoder ')
    types = {'elman': nn.RNN, 'lstm': nn.LSTM, 'gru': nn.GRU}
    output_layers = {'last': LambdaLast, 'avg': LambdaAvg}
    try:
      self.output_layer = output_layers[output_fn]()
    except:
      raise ValueError(f"wrong type '{output_fn}', must be in {list(output_layers.keys())}")

    try:
      rec_module = types[rnn_type]
    except:
      valid_types = list(types.keys())
      raise ValueError(f"wrong type '{rnn_type}', must be in {valid_types}")
    self.rec_module = rec_module(input_size=input_size, hidden_size=hidden_size,
                                 bidirectional=True, batch_first=True,
                                 num_layers=num_layers)
    
  def forward(self, x):
    x, _ = self.rec_module(x)
    x = self.output_layer(x)
    return x


class BagOfVectorsEncoder(torch.nn.Module):

  def __init__(self):
    super().__init__()

  def forward(self, x):
    # TODO: check if the mean is computed on the right axis
    return torch.mean(x, dim=1)


class MLPEncoder(torch.nn.Module):

  def __init__(self, ws_in, w_out, layers=None, name='mlp'):
    super().__init__()
    # the builiding of the network is postponed until input's size is known
    self.ws_in = ws_in
    self.name = name
    self.__build_mlps(w_out, layers)

  def forward(self, x):
    batch_size, max_tokens, embedding_dim = x.shape
    in_features = max_tokens * embedding_dim
    x = torch.reshape(x, (batch_size, in_features))
    if in_features == self.ws_in[0]:
        x = self.evidence_mlp(x)
    elif in_features == self.ws_in[1]:
        x = self.claim_mlp(x)
    else:
        raise ValueError(f'wrong input shape {x.shape}, must be in {self.ws_in}')
    return x

  def __build_mlps(self, w_out, layers):
    if layers is None:
        layers = [w_out, w_out]
    else:
        layers = [w_out] + layers

    self.evidence_mlp = nn.Sequential()
    self.evidence_mlp.add_module(f'{self.name}_fc1', nn.Linear(in_features=self.ws_in[0], out_features=layers[0]))
    for i, (w_in, w_out) in enumerate(zip(layers[:-1], layers[1:])):
      self.evidence_mlp.add_module(f'{self.name}_ReLU', nn.ReLU())
      self.evidence_mlp.add_module(f'{self.name}_fc{i+2}', nn.Linear(in_features=w_in, out_features=w_out))

    self.claim_mlp = nn.Sequential()
    self.claim_mlp.add_module(f'{self.name}_fc1', nn.Linear(in_features=self.ws_in[1], out_features=layers[0]))
    for i, (w_in, w_out) in enumerate(zip(layers[:-1], layers[1:])):
      self.claim_mlp.add_module(f'{self.name}_ReLU', nn.ReLU())
      self.claim_mlp.add_module(f'{self.name}_fc{i+2}', nn.Linear(in_features=w_in, out_features=w_out))

In [ ]:
class FactChecker(torch.nn.Module):

  def __init__(self, embedding_matrix, encoder, merger, ws_in=None, rnn_type=None, rnn_output=None, rec_size=1, hid_size=50, n_layers_classifier=3, cosine_similarity=False):
    """
      A recurrent network performing Neural Language Inference (Fact Checking).
      Params:
        embedding_matrix: the embedding matrix for word embedding
        encoder: [rnn, mlp, bag], the encoder to compute the sentence embedding
        merger: [concatenation, sum, mean], the multi-input merging strategy
        n_layers_classifier: int, the number of layers in the classifier
        RNNEncoder params, only relevant if encoder==rnn:
          rnn_type: [elman, lstm, gru], the RNN architecure used in the encoder
          rnn_output: [last, avg], the function to compute the sentence encoding from the RNN hidden states
          rec_size: int, the number of layers in the rnn
          hid_size: int, the hidden size of the rnn


    """
    super().__init__()
    self.hid_size = hid_size
    self.cosine_similarity = cosine_similarity

    # Word embedding
    emb_size = embedding_matrix.shape[1]
    self.emb_layer = nn.Embedding.from_pretrained(torch.as_tensor(embedding_matrix))

    # Sentence embedding
    if encoder == 'rnn':
      self.encoder = RNNEncoder(emb_size, hid_size, rec_size, rnn_type=rnn_type, output_fn=rnn_output)
    elif encoder == 'mlp':
      self.encoder = MLPEncoder(ws_in=ws_in, w_out=100)
    elif encoder == 'bag':
      self.encoder = BagOfVectorsEncoder()
    else:
      raise ValueError(f"Wrong encoder '{encoder}', must be in ['rnn', 'mlp', 'bag']")

    # Merging
    merging_strategies = {
        'concatenation': LambdaConcatenation,
        'sum': LambdaSum,
        'mean': LambdaMean
    }
    try:
      merging_layer = merging_strategies[merger]
    except:
      valid_strategies = list(merging_strategies.keys())
      raise ValueError(f"wrong type '{merger}', must be in {valid_strategies}")
    self.merger = merging_layer()

    # Classifier
    classifier_in = self.hid_size * 2 if encoder=='rnn' else 100
    if merger == 'concatenation':
      classifier_in *= 2
    if self.cosine_similarity:
      classifier_in += 1
    self.classifier = get_binary_classifier('classifier', w_in=classifier_in, w_hidden=classifier_in//2, n_layers=n_layers_classifier)

  def forward(self, claim, evidence, debug=False):
    # Word embedding
    claim = self.emb_layer(claim).float()
    evidence = self.emb_layer(evidence).float()
    if debug:
      print("After word embedding")
      print(f"\tclaim.shape: {claim.shape}")
      print(f"\tevidence.shape: {evidence.shape}")
    # Sentence embedding
    claim = self.encoder(claim)
    evidence = self.encoder(evidence)
    if debug:
      print("After phrase encoding")
      print(f"\tclaim.shape: {claim.shape}")
      print(f"\tevidence.shape: {evidence.shape}")
    # Merging
    merged_data = self.merger(claim, evidence)
    if debug:
      print("After merging")
      print(f"\tmerged_data.shape: {merged_data.shape}")
    #Cosine Similarity
    if self.cosine_similarity:
      cs = torch.nn.functional.cosine_similarity(claim,evidence, dim=1)
      cs = torch.unsqueeze(cs, dim=1)
      merged_data = torch.cat((merged_data, cs), dim=1)
    # Classifying
    output = self.classifier(merged_data)

    return output

# Training

In [ ]:
def training_step(model, optimizer, loss_fn, data_loader, device):
  model.train()
  log_dict = {'train/loss': [], 'train/accuracy': [], 'train/precision': [], 'train/recall': [], 'train/f1': []}

  for (claims, evidences, labels) in data_loader:
    # forward
    claims = claims.to(device)
    evidences = evidences.to(device)
    labels = labels.to(device)

    outputs = model(claims, evidences)
    outputs = outputs.view(outputs.size(0))
    loss = loss_fn(outputs, labels)
    loss_value = loss.item()

    if not math.isfinite(loss_value):
      print(f"Loss is {loss_value}, stopping training")
      exit(1)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        preds = torch.sigmoid(outputs).round()
        labels, preds = labels.cpu().numpy(), preds.cpu().numpy()
        acc = ((labels == preds).sum() / labels.size).item()
        prec = precision_score(labels, preds)
        rec = recall_score(labels, preds)
        f1 = f1_score(labels, preds)


    log_dict['train/loss'].append(loss_value)
    log_dict['train/accuracy'].append(acc)
    log_dict['train/precision'].append(prec)
    log_dict['train/recall'].append(rec)
    log_dict['train/f1'].append(f1)

  return log_dict


def evaluate(model, loss_fn, data_loader, device):
  """
    Evaluate model on the given dataloader.
    Parameters:
      model: torch.nn.Module to evaluate
      loss_fn: torch.nn criterion to use to compute loss, given outputs and targets
      data_loader: torch.utils.data.DataLoader
      device: torch.device where evaluation is performed
    Returns log dict {'valid/loss' : mean loss, 'valid/{metric}': mean metric}
  """
  model.eval()
  assert len(data_loader) == 1 # must be a single batch
  split = 'valid' # TODO implement for test set

  with torch.no_grad():
    data = next(iter(data_loader))
    data.to(device)

    # Variables for majority voting
    vote_labels = []
    vote_preds = []
    # Variable for standard
    loss_value = []
    std_labels = []
    std_preds = []
    for claims, evidences, labels in data:
      outputs = model(claims, evidences)
      outputs = outputs.view(outputs.size(0))
      preds = torch.sigmoid(outputs).round()
      labels, preds = labels.cpu().numpy(), preds.cpu().numpy()
      # Majority voting
      true_label = np.mean(labels).round()
      predicted_label = np.mean(preds).round()
      vote_labels.append(true_label)
      vote_preds.append(predicted_label)
      # Standard
      loss_value += loss_fn(outputs, labels)
      std_labels += labels
      std_preds += preds
    # Majority voting
    vote_acc = ((vote_labels == vote_preds).sum() / data_loader.batch_size).item()
    vote_prec = precision_score(vote_labels, vote_preds)
    vote_rec = recall_score(vote_labels, vote_preds)
    vote_f1 = f1_score(vote_labels, vote_preds)
    # Standard
    acc = ((std_labels == std_preds).sum() / data_loader.batch_size).item()
    prec = precision_score(std_labels, std_preds)
    rec = recall_score(std_labels, std_preds)
    f1 = f1_score(std_labels, std_preds)
  log_dict = {f'{split}/loss': np.mean(loss_value),
              f'{split}/accuracy': std_acc,
              f'{split}/precision': std_prec,
              f'{split}/recall': std_rec,
              f'{split}/f1': std_f1,
              f'{split}/vote_accuracy': vote_acc,
              f'{split}/vote_precision': vote_prec,
              f'{split}/vote_recall': vote_rec,
              f'{split}/vote_f1': vote_f1}
  return log_dict


In [ ]:
DEFAULT_PARAMS = {
          'encoder': 'rnn',
          'merger': 'concatenation',
          'rnn_type': 'elman',
          'rnn_output': 'last',
          'rec_size': 1,
          'hid_size': 50,
          'cosine_similarity': False
      }
def train(optimizer, lr, device, n_epochs, verbose, batch_size, test=False, tags=None, **model_params):
  model_params = {**DEFAULT_PARAMS, **model_params}

  cfg_dict = {'epochs': n_epochs, 'batch_size': batch_size, 'optimizer': optimizer, 'lr': lr, 'params': model_params}

  wandb.login(key=utils.get_wandbkey())
  run = wandb.init(project="assignment-two", entity="nlpetroni", reinit=True, config=cfg_dict, tags=tags)
  wandb.define_metric("train_step")
  wandb.define_metric("epoch")
  wandb.define_metric('train/loss', step_metric="train_step", summary="min")
  wandb.define_metric("train/accuracy", step_metric="train_step", summary="max")
  wandb.define_metric("valid/loss", step_metric="epoch", summary="min")
  wandb.define_metric("valid/accuracy", step_metric="epoch", summary="max")
  wandb.define_metric("valid/precision", step_metric="epoch", summary="max")
  wandb.define_metric("valid/recall", step_metric="epoch", summary="max")
  wandb.define_metric("valid/f1", step_metric="epoch", summary="max")

  if model_params['encoder'] == 'mlp':
      # with MLP encoder, every sequence must have the same length, thus we must pad at the dataset level
      padding = True
      collate_fn = to_tensor_collate
  else:
      # with other encoders, pad each batch independently
      padding = False
      collate_fn = pad_batch_collate

  vocabulary, embedding_matrix, dataset = load_data(padding=padding)
  train_set, valid_set, test_set = dataset
  train_ds = Dataset(train_set['Claim'], train_set['Evidence'], train_set['Label'])
  valid_ds = Dataset(valid_set['Claim'], valid_set['Evidence'], valid_set['Label'])
  test_ds = Dataset(test_set['Claim'], test_set['Evidence'], test_set['Label'])
  train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
  valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=len(valid_ds), collate_fn=collate_fn)
  test_dl = torch.utils.data.DataLoader(test_ds, batch_size=len(test_ds), collate_fn=collate_fn)
  if model_params['encoder'] == 'mlp':
      model_params['ws_in'] = (len(train_set.loc[0, 'Evidence'] * embedding_matrix.shape[1]), len(train_set.loc[0, 'Claim'] * embedding_matrix.shape[1]))

  model = FactChecker(embedding_matrix, **model_params)
  model.to(device)
  wandb.watch(model, log_graph=True)
  if verbose:
    print(summary(model))

  params = [p for p in model.parameters() if p.requires_grad]
  if optimizer == 'rmsprop':
    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, momentum=0.5, weight_decay=0)
  elif optimizer == 'adam':
    optimizer = torch.optim.Adam(params, lr=lr, betas=(0.9, 0.999), weight_decay=0)
  else:
    raise ValueError(f'wrong optim {optimizer}, either rmsprop or adam')

  loss = nn.BCEWithLogitsLoss()
  valid_loss = nn.BCEWithLogitsLoss(reduction='none')
  train_step = 0
  print('STARTING TRAINING')
  print('Evaluation metrics are computed on validation set\n')
  print(f'EPOCHS | TRAIN LOSS | VALID LOSS | ACCURACY | PRECISION | RECALL |  F1  | V_ACC | V_PREC | V_REC | V_F1')

  for epoch in range(n_epochs):
    log_dict = training_step(model, optimizer, loss, train_dl, device)
    if not test:
      log_dict.update(evaluate(model, valid_loss, valid_dl, device))
      for batch_loss, batch_acc, batch_f1 in zip(log_dict['train/loss'], log_dict['train/accuracy'], log_dict['train/f1']):
        wandb.log({'train_step': train_step, 'epoch': epoch, 'train/loss': batch_loss, 'train/accuracy': batch_acc,
                   'train/f1': batch_f1})
        train_step += 1
      wandb.log({'epoch': epoch, 'valid/loss': log_dict['valid/loss'],
                 'valid/accuracy': log_dict['valid/accuracy'],
                 'valid/precision': log_dict['valid/precision'],
                 'valid/recall': log_dict['valid/recall'],
                 'valid/f1': log_dict['valid/f1'],
                 'valid/vote_accuracy': log_dict['valid/vote_accuracy'],
                 'valid/vote_precision': log_dict['valid/vote_precision'],
                 'valid/vote_recall': log_dict['valid/vote_recall'],
                 'valid/vote_f1': log_dict['valid/vote_f1']})
      print(f'[{epoch:02d}/{n_epochs:02d}]| {np.mean(log_dict["train/loss"]):10.3f} | {log_dict["valid/loss"]:10.3f} | {log_dict["valid/accuracy"]:8.3f} | {log_dict["valid/precision"]:9.3f} | {log_dict["valid/recall"]:6.3f} | {log_dict["valid/f1"]:4.3f} | {log_dict["valid/vote_accuracy"]:5.3f} | {log_dict["valid/vote_precision"]:6.3f} | {log_dict["valid/vote_recall"]:5.3f} | {log_dict["valid/vote_f1"]:4.3f}')
  if test:
    log_dict = evaluate(model, loss, test_dl, device)
    wandb.log()

  run.finish()
  return model

In [ ]:
def GridSearch(parameters, optimizer_name='adam', device=None, verbose=True):
  for rnn_type in parameters['rnn_type']:
    for lr in parameters['lr']:
      for batch_size in parameters['batch_size']:
        for num_layer_class in parameters['num_layer_class']:
          for num_layer_rnn in parameters['num_layer_rnn']:
            train(optimizer_name=optimizer_name, lr=lr, device=device, n_epochs=20, verbose=verbose, batch_size=batch_size,
                encoder='rnn', merger='concatenation', rnn_type=rnn_type, rnn_output='last', hid_size=50 , n_layers_classifier=num_layer_class, rec_size=num_layer_rnn)

# Hyperparameters tuning

## Sentence encoding
### Last state of a RNN

In [ ]:
out_dir = './models'
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
lr = 1e-3
epochs = 20
batch_size = 512
rec_size = 2
hid_size = 50
tag = 'encoding choice'

In [ ]:
# elman
model = train(optimizer='adam', lr=lr, device=device, n_epochs=epochs, verbose=False, tags=[tag],
                         batch_size=batch_size, encoder='rnn', merger='concatenation', rnn_type='elman',
                         rnn_output='last', rec_size=rec_size, hid_size=hid_size, cosine_similarity=False)

torch.save(model.state_dict(), os.path.join(out_dir, 'rnn_last.pth'))

In [ ]:
# lstm
model = train(optimizer='adam', lr=lr, device=device, n_epochs=epochs, verbose=False, tags=[tag],
                        batch_size=batch_size, encoder='rnn', merger='concatenation', rnn_type='lstm',
                        rnn_output='last', rec_size=rec_size, hid_size=hid_size, cosine_similarity=False)
torch.save(model.state_dict(), os.path.join(out_dir, 'lstm_last.pth'))

In [ ]:
# gru
model = train(optimizer='adam', lr=lr, device=device, n_epochs=epochs, verbose=False, tags=[tag],
                       batch_size=batch_size, encoder='rnn', merger='concatenation', rnn_type='gru',
                       rnn_output='last', rec_size=rec_size, hid_size=hid_size, cosine_similarity=False)
torch.save(model.state_dict(), os.path.join(out_dir, 'gru_last.pth'))

### Average of all the output states of a RNN

In [ ]:
# elman
model = train(optimizer='adam', lr=lr, device=device, n_epochs=epochs, verbose=False, tags=[tag],
                        batch_size=batch_size, encoder='rnn', merger='concatenation', rnn_type='elman',
                        rnn_output='avg', rec_size=rec_size, hid_size=hid_size, cosine_similarity=False)
torch.save(model.state_dict(), os.path.join(out_dir, 'rnn_avg.pth'))

In [ ]:
# lstm
model = train(optimizer='adam', lr=lr, device=device, n_epochs=epochs, verbose=False, tags=[tag],
                       batch_size=batch_size, encoder='rnn', merger='concatenation', rnn_type='lstm',
                       rnn_output='avg', rec_size=rec_size, hid_size=hid_size, cosine_similarity=False)
torch.save(model.state_dict(), os.path.join(out_dir, 'lstm_avg.pth'))

In [ ]:
# gru
model = train(optimizer='adam', lr=lr, device=device, n_epochs=epochs, verbose=False, tags=[tag],
                      batch_size=batch_size, encoder='rnn', merger='concatenation', rnn_type='gru',
                      rnn_output='avg', rec_size=rec_size, hid_size=hid_size, cosine_similarity=False)
torch.save(model.state_dict(), os.path.join(out_dir, 'gru_avg.pth'))

### MLP layer

In [ ]:
model = train(optimizer='adam', lr=lr, device=device, n_epochs=epochs, verbose=False,  tags=[tag],
              batch_size=batch_size, encoder='mlp', merger='concatenation', cosine_similarity=False)
torch.save(model.state_dict(), os.path.join(out_dir, 'mlp.pth'))

### Bag of vectors

In [ ]:
model = train(optimizer='adam', lr=lr, device=device, n_epochs=epochs, verbose=False,  tags=[tag],
              batch_size=batch_size, encoder='bag', merger='concatenation', cosine_similarity=False)
torch.save(model.state_dict(), os.path.join(out_dir, 'bow.pth'))

The best sentence embedding is